In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import nibabel as nib
import math
import os
from scipy.stats import multivariate_normal
from scipy.signal import fftconvolve
from skimage.transform import resize
from scipy.ndimage import affine_transform

In [ ]:
# Gaussian PSF standard deviation per axis.
sigmax = 2.
sigmay = 2.
sigmaz = 2.5

# Selection of axis for truncation.
axis = 2

# Downscaling factor per axis.
scalex = 2
scaley = 2
scalez = 2

# Standard deviation of additive white Gaussian noise.
wgnsigma = 0

# Covariance matrix of Gaussian PSF.
PSF_Cov = np.array([[sigmax**2,0,0],
                  [0,sigmay**2,0],
                  [0,0,sigmaz**2]])

# PSF kernel axis-wise length.
length = 4*np.ceil(np.sqrt(np.amax(PSF_Cov))) + 1

# Since nominal spatial resolution in ground truth images is approximately 1mm isotropic, 
# the axis-wise distance between the centers of two consecutive voxels with respect to x, 
# y, and z axes is either 0 or 1mm. Hence, sampling the Gaussian pdf at the respective 
# discrete grid, as outlined in [1], and applying normalization yields the Gaussian kernel for PSF.
# [1] https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.multivariate_normal.html
f = np.floor(length/2)
x, y, z = np.mgrid[-f:(f+1), -f:(f+1), -f:(f+1)]
pos = np.empty(x.shape + (3,))
pos[:, :, :, 0] = x
pos[:, :, :, 1] = y
pos[:, :, :, 2] = z
mean = [0, 0, 0]
rv = multivariate_normal(mean, PSF_Cov)

# The resulting kernel is normalized to defuce the PSF kernel.
PSF = rv.pdf(pos)/np.sum(rv.pdf(pos))

# Truncation of one axis (always the slice-select axis)
truncation = np.floor(length/3).astype(np.int)

if axis == 0:
    PSF[:truncation,:,:] = 0
    PSF[-truncation:,:,:] = 0
elif axis == 1:
    PSF[:,:truncation,:] = 0
    PSF[:,-truncation:,:] = 0
else:
    PSF[:,:,:truncation] = 0
    PSF[:,:,-truncation:] = 0

# Re-normalization of the PSF kernel.
PSF = PSF/np.sum(PSF)

# Define the input path for ground truth images.
input_path = 'D:/Data/IXI-T1-IOP/'
# Define the save path for randomly rotated ground truth images.
save_path_gt = 'D:/Data/IXI-T1-IOP-GT/'
# Define the save path for the interpolated images.
save_path_interp = 'D:/Data/IXI-T1-IOP-SIG-2225-SC-222-N-0/'

for root, dirs, filenames in os.walk(input_path):
    for f in filenames:
        print(f)
        fullpath = os.path.join(input_path, f)
        img = nib.load(fullpath)
        data = np.rot90(img.get_data()).astype(np.float64)
        
        # m_max pertains to the number of images which will be created based 
        # on the same ground truth image of a subject.
        m_max = 2
        for m in range(0,m_max):
            print(m)
            
            # The ground truth image is undergone random rotation.
            theta_x = np.random.uniform(0,90)
            theta_y = np.random.uniform(0,90)
            theta_z = np.random.uniform(0,90)
            
            theta_x = theta_x/180.0*math.pi
            Rx = np.array([[1, 0, 0],[0, np.cos(theta_x), -np.sin(theta_x)],[0, np.sin(theta_x), np.cos(theta_x)]])
            theta_y = theta_y/180.0*math.pi
            Ry = np.array([[np.cos(theta_y), 0, np.sin(theta_y)],[0, 1, 0],[-np.sin(theta_y), 0, np.cos(theta_y)]])
            theta_z = theta_z/180.0*math.pi
            Rz = np.array([[np.cos(theta_z), -np.sin(theta_z), 0],[np.sin(theta_z), np.cos(theta_z), 0], [0, 0, 1]])
            R = np.matmul(np.matmul(Rz, Ry), Rx)
               
            center_ = 0.5 * np.asarray(data.shape, dtype=np.int64)
            c_offset = center_ - center_.dot(R)
            data_rot = affine_transform(data, R.T, c_offset, order=3)

            # The randomly rotated ground truth image is saved.
            new_img = nib.Nifti1Image(data_rot, img.affine, img.header)
            nib.save(new_img, os.path.join(save_path_gt, f[:-7]+'-'+str(m+1)+'.nii.gz'))
            
            # The randomly rotated ground truth is convolved with the PSF kernel. 
            # It is to be noted that the application of PSF is axis-aligned with 
            # respect to the acquisition system and not to the random rotation 
            # applied to the ground truth image.
            data_blurred = fftconvolve(data_rot, PSF, mode='same')
            
            # The blurred image is downsampled.
            data_blurred_downsampled = resize(data_blurred, (int(data.shape[0]/scalex), int(data.shape[1]/scaley), int(data.shape[2]/scalez)), order = 3, mode = 'reflect')
            
            # Noise is added to the blurred and downsampled image.
            whitenoise = np.random.normal(0, wgnsigma, (data_blurred_downsampled.shape[0], data_blurred_downsampled.shape[1], data_blurred_downsampled.shape[2]))
            data_blurred_downsampled_noisy = data_blurred_downsampled + whitenoise
            
            # The blurred, downsampled, and noisy image is interpolated and, 
            # thus, it now matches the size of its respective ground truth image.
            data_blurred_downsampled_noisy_interp = resize(data_blurred_downsampled_noisy, (data.shape[0], data.shape[1], data.shape[2]), order = 3, mode = 'reflect') 
            
            # The interpolated image is saved.
            new_img = nib.Nifti1Image(data_blurred_downsampled_noisy_interp, img.affine, img.header)
            nib.save(new_img, os.path.join(save_path_interp, f[:-7]+'-'+str(m+1)+'.nii.gz'))